# AnthropicToolCallState

## Example: Create TOOL_CALL state using AnthropicToolUseState

```lua
stateDiagram-v2
    INIT --> TOOL_CALL
    TOOL_CALL --> FINAL
```

```mermaid
stateDiagram-v2
    Direction LR
    INIT --> TOOL_CALL
    TOOL_CALL --> FINAL
```


In [1]:
from gai.asm import AsyncStateMachine
from gai.mcp.client import McpAggregatedClient

with AsyncStateMachine.StateMachineBuilder(
    """
    INIT --> TOOL_CALL
    TOOL_CALL--> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "user_message": "What time is it now?",
                    "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                    "mcp_client": {"type": "getter", "dependency": "get_mcp_client"},
                }
            },
            "TOOL_CALL": {
                "module_path": "gai.asm.states",
                "class_name": "AnthropicToolCallState",
                "title": "TOOL_CALL",
                "input_data": {
                    "user_message": {"type": "state_bag", "dependency": "user_message"},
                    "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                    "mcp_client": {"type": "state_bag","dependency": "mcp_client"},
                },
                "output_data": ["streamer", "get_assistant_message"],
            },
            "FINAL": {
                "output_data": ["monologue"],
            },
        },
        get_llm_config=lambda state: {
            "client_type": "anthropic",
            #"model": "claude-opus-4-20250514",
            "model": "claude-sonnet-4-20250514",
            "max_tokens": 32000,
            "temperature": 0.7,
            "top_p": 0.95,
            "tools": True,
        },
        get_mcp_client=lambda state: McpAggregatedClient(["mcp-time"])
    )

## Step 2: INIT --> TOOL_CALL

await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    if (isinstance(chunk,str)):
        print(chunk, end='', flush=True)
print("\n\n")

## Step 4: Print the state history
for message in fsm.state_history[1]["output"]["monologue"].list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

I'll get the current time for you.


1750070576.4111097 User > What time is it now?
1750070579.2760587 Assistant > [{'citations': None, 'text': "I'll get the current time for you.", 'type': 'text'}, {'id': 'toolu_01BKbWGchMW25RMXDkqasYK8', 'input': {'format': 'YYYY-MM-DD HH:mm:ss'}, 'name': 'current_time', 'type': 'tool_use'}]
